In [1]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

Setting up Sandbox...
Done!


In [ ]:
## 1 导入数据
grid用来帮忙管理数据。（sandbox）。

In [2]:
boston_data = grid.search("#boston", "#data")
boston_target = grid.search("#boston", "#target")

In [16]:
# 3 定义模型
n_features = boston_data['alice'][0].shape[1]
n_targets = 1

model = th.nn.Linear(n_features, n_targets)
print(boston_data['alice'][0])

(Wrapper)>[PointerTensor | me:27894392138 -> alice:84849301825]
	Tags: .. #boston_housing #housing _boston_dataset: #boston #data 
	Shape: torch.Size([84, 13])
	Description: .. _boston_dataset:...


In [14]:
# 3 定义多个数据和多个优化器
# Cast the result in BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)
optimizers = {}
for worker in dataset.workers:
    optimizers[worker] = th.optim.Adam(params=model.parameters(),lr=1e-2)

['bob', 'theo', 'jason', 'alice', 'andy', 'jon']


In [17]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=32, shuffle=False, drop_last=False)

In [19]:
# 对分布式数据进行训练
epochs = 1
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        model.send(data.location)
        
        optimizer = optimizers[data.location.id]
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred.view(-1) - target)**2).mean()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 8 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBatch loss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))            
            
    print('Total loss', loss_accum)

Train Epoch: 1 [0/16 (0%)]	Batch loss: 56.270569
Train Epoch: 1 [8/16 (50%)]	Batch loss: 199.320129
Train Epoch: 1 [16/16 (100%)]	Batch loss: 359.854309
Total loss 4278.3408851623535
